## 멜론 가사 수집 (장르별) 정적 스크래핑 

정적 수집시 좋아요는 화면 진입시 동적으로 생성되어 가져올 수 없는 버전 입니다. 

### 1. 환경 설정 

In [13]:
# 1. 필요 라이브러리 추가 
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
from tqdm import tqdm
import time
import random
import datetime

### 파라미터 세팅 
#### url 특징 
 *  https://www.melon.com/genre/song_list.htm?gnrCode=GN0500
    * gnrCode = 장르별 코드 
    * GN0100 발라드 / GN0200 댄스 / GN0300 랩·힙합 / GN0400 R&b·Soul / GN0500 인디음악 / GN0600 록·메탈 / GN0700 트로트 / GN0800 포크·블루스 
    * GN0900 POP / GN1000 록·메탈 / GN1100 일렉트로니카 / GN1200 랩·힙합 / GN1300 R&b·Soul / GN1400 포크·블루스·컨트리
 * https://www.melon.com/song/detail.htm?songId=38427225
    * songId= 곡 ID 

In [14]:
# 장르 메뉴 정의
melon_genres = {                  # 국내 장르 
    "발라드": "GN0100",
    "랩/힙합": "GN0300",
    "R&B/Soul": "GN0400",
    "인디음악": "GN0500",
    "트로트": "GN0600",
    "해외 록/메탈": "GN1000",
    "해외 R&B/Soul": "GN1300"
}

### 2. 데이터 불러오기 

In [15]:
# # 가사 수집 함수  

# def get_lyrics(song_list) : 

#     # 데이터프레임 초기화
#     # columns = ['chartDate', 'rank', 'title', 'singer', 'album_name', 'release_date', 'genre', 'lyric', 'composer', 'lyricist', 'arranger']
#     # 곡 제목 title / 가사 lylics / 아티스트 artist / 장르 ganre / 발매일 date / 좋아요 like

#     columns = ['title', 'artist', 'ganre', 'release_date', 'like_cnt', 'lylics']
#     song_data = pd.DataFrame(columns=columns)

#     # tqdm 라이브러리로 진행 상황 바 표시
#     for i, meta in tqdm(enumerate(song_list, 1), total=len(song_list), desc="Processing songs"):
#         rank = i

#         # 모든 곡 정보를 포함하는 요소 선택
#         songs = meta.select('.wrap_song_info')

#         # 각 곡 정보에서 곡 제목 추출
#         for song in song_list:
#             try: 
#                 title_element = song.select_one('.ellipsis.rank01 a')  # 곡 제목 선택
#                 if title_element:  # 요소가 존재할 경우
#                     # song_titles.append(title_element.text.strip())  # 제목을 리스트에 추가
#                     title = title_element.text.strip()
#                     href = title_element['href']  # href 속성 가져오기
#                     # 정규 표현식을 사용하여 곡 ID 추출
#                     match = re.search(r"playSong\('(\d+)',(\d+)\)", href)
#                     # if match:
#                     song_id = match.group(2)  # 두 번째 그룹이 곡 ID
#                     song_url = 'https://www.melon.com/song/detail.htm?songId=' + song_id

#                     response = requests.get(song_url, params=params, headers=headers)
#                     soup = BeautifulSoup(response.text, 'html.parser')

#                     # 가수
#                     singer_html = soup.select('.wrap_info .artist a')
#                     singer_s = ', '.join([html['title'] for html in singer_html if html['title']]) if singer_html else 'Various Artists'

#                     # 앨범명
#                     # album_name = soup.select('.list dd')[0].get_text(strip=True)

#                     # 발매날짜
#                     release_date = soup.select('.list dd')[1].get_text(strip=True)

#                     # 장르
#                     genre = soup.select('.list dd')[2].get_text(strip=True)

#                     # 좋아요 
#                     # <span id="d_like_count" class="cnt">44</span>
#                     # like_cnt = soup.select('.cnt').get_text(strip=True)

#                     # 예시 코드
#                     like_count_element = meta.select_one('#d_like_count')  # ID로 요소 선택
#                     if like_count_element:  # 요소가 존재할 경우
#                         like_cnt = like_count_element.text.strip()  # 텍스트 가져오기 및 공백 제거
#                     else:
#                         like_cnt = 0


#                     # 가사
#                     lyric = '없음'
#                     lyric_html = soup.select_one('.section_lyric .wrap_lyric .lyric')
#                     if lyric_html:
#                         lyric = lyric_html.get_text(strip=True, separator='\n')

#                     row = pd.Series([title, singer_s, genre, release_date, like_cnt,  lyric], index=song_data.columns)
#                     song_data = pd.concat([song_data, pd.DataFrame([row])], ignore_index=True)

#                     # 1초에서 5초 사이의 랜덤한 시간 선택
#                     random_sleep_time = random.uniform(1, 5)
#                     time.sleep(random_sleep_time)  # IP 차단 방지용 랜덤한 시간 동안 대기
#             except Exception as e:
#                 print(f"오류 발생: {e} - {title if 'title' in locals() else 'Unknown'} (건너뜀)")
#                 continue  # 오류 발생 시 다음 곡으로 넘어감
            
#     return song_data

In [16]:
# def get_lyrics(song_list):
#     # 컬럼 정의
#     columns = ['title', 'artist', 'genre', 'release_date', 'like_cnt', 'lyrics']
#     song_data_list = []  # 리스트로 저장 후 한 번에 DataFrame 변환

#     for i, meta in tqdm(enumerate(song_list, 1), total=len(song_list), desc="Processing songs"):
#         try:
#             # 곡 제목 가져오기
#             title_element = meta.select_one('.ellipsis.rank01 a')
#             if not title_element:
#                 continue  # 제목이 없으면 스킵

#             title = title_element.text.strip()
#             href = title_element['href']

#             # 곡 ID 추출
#             match = re.search(r"playSong\('(\d+)',(\d+)\)", href)
#             if not match:
#                 continue  # ID를 찾을 수 없으면 스킵

#             song_id = match.group(2)
#             song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'

#             # HTTP 요청
#             response = requests.get(song_url, params=params, headers=headers)
#             if response.status_code != 200:
#                 print(f"⚠️ {title} - 페이지 요청 실패")
#                 continue  # 요청 실패 시 스킵

#             soup = BeautifulSoup(response.text, 'html.parser')

#             # 가수
#             singer_html = soup.select('.wrap_info .artist a')
#             singer_s = ', '.join([html['title'] for html in singer_html if html.get('title')]) if singer_html else 'Various Artists'

#             # 앨범/발매일/장르 정보 추출
#             song_info = soup.select('.list dd')

#             release_date = song_info[1].get_text(strip=True) if len(song_info) > 1 else "Unknown"
#             genre = song_info[2].get_text(strip=True) if len(song_info) > 2 else "Unknown"

#             # 좋아요 수
#             like_count_element = soup.select_one('#d_like_count')
#             like_cnt = like_count_element.text.strip() if like_count_element else '0'

#             # 가사
#             lyric = '없음'
#             lyric_html = soup.select_one('.section_lyric .wrap_lyric .lyric')
#             if lyric_html:
#                 lyric = lyric_html.get_text(strip=True, separator='\n')

#             # 데이터 리스트에 추가
#             song_data_list.append([title, singer_s, genre, release_date, like_cnt, lyric])

#             # 랜덤 대기 (1~5초)
#             time.sleep(random.uniform(1, 5))

#         except Exception as e:
#             print(f"❌ 오류 발생: {e} - {title if 'title' in locals() else 'Unknown'} (건너뜀)")
#             continue  # 오류 발생 시 다음 곡으로 넘어감

#     # 리스트를 DataFrame으로 변환
#     song_data = pd.DataFrame(song_data_list, columns=columns)
    
#     return song_data


In [17]:


def get_lyrics(song_list):
    columns = ['title', 'artist', 'genre', 'release_date', 'like_cnt', 'lyrics']
    song_data_list = []

    for i, meta in tqdm(enumerate(song_list, 1), total=len(song_list), desc="Processing songs"):
        try:
            title_element = meta.select_one('.ellipsis.rank01 a')
            if not title_element:
                print(f"⚠️ [{i}] 제목 없음 - 스킵")
                continue

            title = title_element.text.strip()
            href = title_element['href']

            match = re.search(r"playSong\('(\d+)',(\d+)\)", href)
            if not match:
                print(f"⚠️ [{i}] {title} - 곡 ID 없음 - 스킵")
                continue

            song_id = match.group(2)
            song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'

            response = requests.get(song_url, params=params, headers=headers)
            if response.status_code != 200:
                print(f"⚠️ [{i}] {title} - 요청 실패 (Status Code: {response.status_code}) - 스킵")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')

            # 가수
            singer_html = soup.select('.wrap_info .artist a')
            if not singer_html:
                singer_s = 'Various Artists'
            else:
                singer_s = ', '.join([html['title'] for html in singer_html if html.get('title')])

            # 발매일 & 장르 처리
            song_info = soup.select('.list dd')
            release_date = song_info[1].get_text(strip=True) if len(song_info) > 1 else "Unknown"
            genre = song_info[2].get_text(strip=True) if len(song_info) > 2 else "Unknown"

            # 좋아요 수
            like_count_element = soup.select_one('#d_like_count')
            like_cnt = like_count_element.text.strip() if like_count_element else '0'

            # 가사
            lyric_html = soup.select_one('.section_lyric .wrap_lyric .lyric')
            lyric = lyric_html.get_text(strip=True, separator='\n') if lyric_html else "없음"

            song_data_list.append([title, singer_s, genre, release_date, like_cnt, lyric])

            print(f"✅ [{i}] {title} - 수집 성공")

            time.sleep(random.uniform(1, 3))

        except Exception as e:
            print(f"❌ [{i}] {title if 'title' in locals() else 'Unknown'} - 오류 발생: {e} (건너뜀)")
            continue

    song_data = pd.DataFrame(song_data_list, columns=columns)
    
    return song_data


In [18]:
import os
os.getcwd()

"c:\\Wanted\\02_ProjectCollections\\25.01_프로젝트#2_'BE MY MUSE' 작사 공모전\\(수집코드)"

In [19]:
# 파일로 저장 
def make_to_csv (song_data,genre) : 
    #데이터 프레임 저장
    address = '../(Data)'

    # 현재 시간 가져오기
    now = datetime.datetime.now()
    # 시간 형식 지정 (예: '2025-01-15_14-30-00')
    timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

    # 파일 이름 생성
    file_name = f"melon_{genre}_{timestamp}.csv"

    # song_data.to_csv(address, index=False, encoding='utf-8-sig')
    song_data.to_csv(path_or_buf=address+file_name)

    return print(f"{file_name}이 저장되었습니다.")

### 실행부 

In [20]:
# # request 를 사용하여 데이터 수집할 화면 가져오기 
# headers = {
#     'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
#                 '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')
# }

# gnr_url = "https://www.melon.com/genre/song_list.htm"

# params = dict()
# curr_genre = '' 
# for key,value in melon_genres.items(): 
#     curr_genre = key
#     params['gnrCode'] = value
#     response = requests.get(gnr_url, params=params, headers=headers)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     song_list = soup.select('.wrap_song_info')
    
#     song_data = get_lyrics(song_list)

# # 사용자 입력이 들어올 때까지 대기
# while song_data is None:
#     pass  # 계속 대기

# if song_data is not None : 
#     make_to_csv (song_data,curr_genre)

In [21]:
headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')
}

gnr_url = "https://www.melon.com/genre/song_list.htm"
params = {}

try:
    for curr_genre, genre_code in melon_genres.items():
        try:
            params['gnrCode'] = genre_code
            response = requests.get(gnr_url, params=params, headers=headers)

            # if response.status_code != 200:
            #     print(f"⚠️ {curr_genre} 장르 페이지 요청 실패 (Status Code: {response.status_code})")
            #     continue  # 다음 장르로 넘어가기

            soup = BeautifulSoup(response.text, 'html.parser')
            song_list = soup.select('.wrap_song_info')

            # if not song_list:
            #     print(f"⚠️ {curr_genre} 장르에서 곡을 찾을 수 없음.")
            #     continue  # 다음 장르로 넘어가기

            song_data = get_lyrics(song_list)

            if song_data is not None and not song_data.empty:
                make_to_csv(song_data, curr_genre)
            else:
                print(f"⚠️ {curr_genre} 장르의 데이터가 존재하지 않음.")

        except Exception as e:
            print(f"❌ {curr_genre} 장르 처리 중 오류 발생:", e)
            make_to_csv(song_data, curr_genre)
            continue  # 예외 발생 시 다음 장르 처리

except Exception as e:
    print("❌ 전체 처리 중 예상치 못한 오류 발생:", e)
    make_to_csv(song_data, curr_genre)


Processing songs:   0%|          | 0/100 [00:00<?, ?it/s]

✅ [1] 가끔 우리가 아직 사랑하는 상상을 해 - 수집 성공


Processing songs:   1%|          | 1/100 [00:03<05:08,  3.12s/it]

⚠️ [2] 제목 없음 - 스킵
✅ [3] 사랑이 남겨준 마지막 선물이었을 테니까 (Feat. 윤도) - 수집 성공


Processing songs:   3%|▎         | 3/100 [00:05<02:31,  1.57s/it]

...

Processing songs:  97%|█████████▋| 97/100 [01:42<00:03,  1.15s/it]

⚠️ [98] 제목 없음 - 스킵
✅ [99] 왜 이래 (Feat. 신해솔) - 수집 성공


Processing songs: 100%|██████████| 100/100 [01:43<00:00,  1.04s/it]


⚠️ [100] 제목 없음 - 스킵
❌ 랩/힙합 장르 처리 중 오류 발생: Cannot save file into a non-existent directory: '..\(Data)melon_랩'
❌ 전체 처리 중 예상치 못한 오류 발생: Cannot save file into a non-existent directory: '..\(Data)melon_랩'


OSError: Cannot save file into a non-existent directory: '..\(Data)melon_랩'